# Задача 1: сравнение предложений

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

    кошки (животные)
    UNIX-утилита cat для вывода содержимого файлов
    версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.
sentences.txt

Выполните следующие шаги:

    Скачайте файл с предложениями (sentences.txt).
    Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
    Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
    Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
    Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
    Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
    Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).
    Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.

In [46]:
import re

import numpy as np
import scipy.spatial.distance

In [3]:
with open('sentences.txt', 'r') as f:
    sentences = [l.lower().strip() for l in f]

In [39]:
tokenized_sentences = [list(filter(lambda t: t, re.split('[^a-z]', s))) for s in sentences]

In [40]:
words = []
words_idx = {}

for toks in tokenized_sentences:
    for t in toks:
        if t not in words_idx:
            words_idx[t] = len(words)
            words.append(t)

In [36]:
tf = np.zeros((len(sentences), len(words)))

In [37]:
tf.shape

(22, 254)

In [41]:
for s, toks in enumerate(tokenized_sentences):
    for t in toks:
        tf[s, words_idx[t]] += 1

In [43]:
scipy.spatial.distance.cosine()

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 1., 1.]])

In [54]:
dists = np.zeros(len(sentences)-1)

for i, v in enumerate(tf[1:]):
    dists[i] = scipy.spatial.distance.cosine(tf[0], v)

In [56]:
np.argpartition(dists, 2)

array([ 5,  3, 20,  1,  4,  0,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19,  2], dtype=int64)

In [57]:
dists

array([0.95275444, 0.86447381, 0.89517152, 0.77708871, 0.94023857,
       0.73273876, 0.92587507, 0.88427249, 0.90550888, 0.83281654,
       0.88047714, 0.83964325, 0.87035926, 0.87401184, 0.94427218,
       0.84063619, 0.9566445 , 0.94427218, 0.88854436, 0.84275727,
       0.82503645])